In [58]:
pip install pandas transformers


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
import pandas as pd

def count_words(text):
    words = text.split()
    return len(words)

def main(input_csv_file, output_csv_file):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(input_csv_file)

    # Calculate the number of words in the comment section and create a boolean mask
    df['word_count'] = df['comment'].apply(count_words)
    mask = df['word_count'] <= 100

    # Filter the DataFrame based on the mask
    df_ytb = df[mask]

    # Drop the 'word_count' column as it's no longer needed
    df_ytb = df_ytb.drop(columns=['word_count'])

    # Reset index to rearrange numbering
    df_ytb = df_ytb.reset_index(drop=True)

    # Write the filtered DataFrame to a new CSV file
    df_ytb.to_csv(output_csv_file, index=False)

    return df_ytb

if __name__ == "__main__":
    input_csv_file = "/content/drive/MyDrive/CSV FILES/ytb_UNSANE.csv"
    output_csv_file = "output.csv"

    df_ytb = main(input_csv_file, output_csv_file)
    print(df_ytb)

    Unnamed: 0                                            comment  \
0            0  99% of people who are watching this video is b...   
1            1  Yeah I’m sorry I can’t watch the show. I’m not...   
2            2  Go ahead and laugh, its not just a movie, beca...   
3            3  i feel like this is what they do to unvaccinat...   
4            4  Jat Pharaoh surprised the hell out of me in th...   
5            5                  Shot on iphone<br><br>By Linda H.   
6            6  Something doesn&#39;t feel right about these c...   
7            7  This was shot on a iPhone 7?<br>Now that’s imp...   
8            8                      Wheewwwwww boy that was tense   
9            9  I wish to be in a psych ward like that. Cooper...   
10          10  I would of just walked away as soon as he said...   
11          12  This film was done on an iPhone 7, that goes t...   
12          13       Looks like an episode of a tv show from 1994   
13          14                    

In [61]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

#df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Kingdom of Us.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(15, 2)


In [62]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '2', 'score': 0.7697238326072693},
 {'label': '0', 'score': 0.7765718698501587},
 {'label': '0', 'score': 0.48437052965164185},
 {'label': '0', 'score': 0.4561692774295807},
 {'label': '2', 'score': 0.49169737100601196},
 {'label': '1', 'score': 0.3565594255924225},
 {'label': '1', 'score': 0.686524510383606},
 {'label': '1', 'score': 0.6785151958465576},
 {'label': '0', 'score': 0.6543704271316528},
 {'label': '2', 'score': 0.9101526737213135},
 {'label': '2', 'score': 0.9056677222251892},
 {'label': '2', 'score': 0.5297500491142273},
 {'label': '1', 'score': 0.5170838236808777},
 {'label': '1', 'score': 0.36957770586013794},
 {'label': '2', 'score': 0.5467610955238342}]

In [63]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,99% of people who are watching this video is b...,2,0.769724
1,Yeah I’m sorry I can’t watch the show. I’m not...,0,0.776572
2,"Go ahead and laugh, its not just a movie, beca...",0,0.484371
3,i feel like this is what they do to unvaccinat...,0,0.456169
4,Jat Pharaoh surprised the hell out of me in th...,2,0.491697
5,Shot on iphone<br><br>By Linda H.,1,0.356559
6,Something doesn&#39;t feel right about these c...,1,0.686525
7,This was shot on a iPhone 7?<br>Now that’s imp...,1,0.678515
8,Wheewwwwww boy that was tense,0,0.654370
9,I wish to be in a psych ward like that. Cooper...,2,0.910153


In [64]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         2           love
1         0        sadness
2         0        sadness
3         0        sadness
4         2           love
5         1            joy
6         1            joy
7         1            joy
8         0        sadness
9         2           love
10        2           love
11        2           love
12        1            joy
13        1            joy
14        2           love


<ipython-input-64-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [65]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_UNSANE_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,99% of people who are watching this video is b...,2,0.769724,love
1,Yeah I’m sorry I can’t watch the show. I’m not...,0,0.776572,sadness
2,"Go ahead and laugh, its not just a movie, beca...",0,0.484371,sadness
3,i feel like this is what they do to unvaccinat...,0,0.456169,sadness
4,Jat Pharaoh surprised the hell out of me in th...,2,0.491697,love
5,Shot on iphone<br><br>By Linda H.,1,0.356559,joy
6,Something doesn&#39;t feel right about these c...,1,0.686525,joy
7,This was shot on a iPhone 7?<br>Now that’s imp...,1,0.678515,joy
8,Wheewwwwww boy that was tense,0,0.654370,sadness
9,I wish to be in a psych ward like that. Cooper...,2,0.910153,love
